# Localization

In [93]:
import numpy as np
from spatialmath.base import skew, qeye
from spatialmath import UnitQuaternion as UQ

# Problem 1

In [94]:
#define function that takes current quaternion, angular velocity, and timestep to produce an estimated quaternion
def fw_step(q_curr, ang_vel, t_step): #ang_vel must be shape (1, 3)
    if ang_vel.shape != (1, 3):
        raise ValueError(f"Expected angular velocity shape (1, 3), got {ang_vel.shape}")
    if q_curr.shape != (1, 4):
        raise ValueError(f"Expected input quaternion shape (1, 4), got {q_curr.shape}")

    #construct the omega Matrix
    omegaM = np.block([
    [0                     ,                 -ang_vel],
    [ang_vel.reshape(3, -1),           -skew(ang_vel)]
    ])

    #compute closed form exponential
    ang_vel_norm = np.linalg.norm(ang_vel)
    e = (np.cos(ang_vel_norm*0.5*t_step) * np.eye(4,4)) + (np.sin(ang_vel_norm*0.5*t_step) * (omegaM/ang_vel_norm))
    
    #construct updated quaternion
    q_update = e @ q_curr.reshape(4,-1)
   
    return q_update.reshape(-1,4)


timestep = 0.25 #1/4 second
omega_x = np.array([[np.pi, 0, 0]])
omega_z = np. array([[0, 0, np.pi]])
q_initial = np.array([qeye()])

#Rotate about the body x-axis for 1/4 s at π rad/s.
q_1 = fw_step(q_initial, omega_x, timestep)
print("Estimated quaternion after first step: ", q_1)

#Rotate about the body z-axis for 1/4 s at π rad/s
q_2 = fw_step(q_1, omega_z, timestep)
print("Estimated quaternion after second step: ", q_2)

#Rotate about the body x-axis for 1/4 s at −π rad/s
q_3 = fw_step(q_2, -omega_x, timestep)
print("Estimated quaternion after third step: ", q_3)

#Rotate about the body z-axis for 1/4 s at −π rad/s
q_4 = fw_step(q_3, -omega_z, timestep)
print("Estimated quaternion after fourth step: ", q_4)

Estimated quaternion after first step:  [[0.92387953 0.38268343 0.         0.        ]]
Estimated quaternion after second step:  [[ 0.85355339  0.35355339 -0.14644661  0.35355339]]
Estimated quaternion after third step:  [[ 0.92387953  0.         -0.27059805  0.27059805]]
Estimated quaternion after fourth step:  [[ 0.95710678  0.10355339 -0.25       -0.10355339]]


 Q1) Report the rotation q(1) at the end of this motion, i.e., after 1s has elapsed. Does it hold that q(1) = q(0)? Discuss why this is true or false.

 q(1) != q(0) for two potential reasons:
 1. In order to get back to intial orientation the rotation backwards through the z-axis would have to come first. Doing a rotation xzxz is not the same as doing xzzx
 2. 0.25 second time step is a large amount of time for our linearization assumption. t_step should be sufficiently small to properly linearize our integration.

# Problem 2